In [28]:
import torch, torchvision

In [14]:
class CNN(torch.nn.Module):
  def __init__(self, input_size: int, hidden1_size: int, hidden_size2: int, output_size: int):
    super().__init__()
    
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

    self.flatten = torch.nn.Flatten()

    self.fc1 = torch.nn.Linear(in_features=32*7*7, out_features=hidden1_size)
    self.fc2 = torch.nn.Linear(in_features=hidden1_size, out_features=hidden_size2)
    
    self.ol = torch.nn.Linear(in_features=hidden_size2, out_features=output_size)
  
  def forward(self, input_data):
    max_pooled_conv1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(torch.nn.functional.relu(self.conv1(input_data)))
    max_pooled_conv2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(torch.nn.functional.relu(self.conv2(max_pooled_conv1)))

    flattened = self.flatten(max_pooled_conv2)

    fc1_out = torch.nn.functional.relu(self.fc1(flattened))
    fc2_out = torch.nn.functional.relu(self.fc2(fc1_out))

    outputs = torch.nn.functional.softmax(self.ol(fc2_out))

    return outputs
    
    

In [37]:
input_data = torch.randint(255, (10000, 28, 28), dtype=torch.uint8)
transormer = torchvision.transforms.Compose([torchvision.transforms.ConvertImageDtype(torch.float32),
                                             torchvision.transforms.Normalize((0.5,), (0.5,))])
transformed_data = transormer(input_data)

train_data, val_data = torch.utils.data.random_split(transformed_data, [8000, 2000])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=10, num_workers=1)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=10, num_workers=1)